In [ ]:
#verrijking van de harstilstand data doen eerst, dat is het allereerste. dan de slechte harstilstanden eruit halen, die dat al 
#volledig gedekt zijn door de ziekenhuizen, want die hoeven toch al niet meer gered te worden. Dan kan je zien welke AED's nuttig zijn en welke niet,
#door die netwerk analyse toe te passen en te zien welke AED's niet verbonden zijn (niet de dichtstbijzijnde) bij een harstilstand zijn. (PAS OP VOOR
#POTENTIELE RANDGEVALLEN, ALS ER BIJVOORBEELD meerdere harstilstanden vlak naast elkaar zijn, kan het zijn dat een AED eigenlijk slecht is maar toch het
#dichtst staat bij een harstilstand, die al gedekt is => HIER REKENING MEE HOUDEN, => OPLOSSING GEEN BIPARTITE GRAPH GEBRUIKEN, JE VERBINDT ALLE 
#HARTSTILSTANDEN ONDER EEN BEPAALDE TIJD MET ALLE AED'S DIE ONDER DIE BEPAALDE TIJD ZIJN, EN DAN ACHTERAF KAN JE BESLISSEN HOE HET ZIT)

#opmerking: er moet sowieso een ziekenwagen langskomen, of er nu een AED wordt gebruikt of niet. Dus die dat dichter dan 3 minuten zijn (of welke tijd
#we ook beslissen te nemen) worden toch al gered, dus daar hoeft simpelweg geen AED te komen. Eigenlijk is een AED sowieso waardevol, het hele ding
#is juist dat de middelen beperkt zijn, en dat die AED's toch gemist kunnen worden. En dat doe je door de cases die al gered worden zo te 
#verwijderen uit de analyse.

In [1]:
#AED's die te dicht bij de ziekenhuizen staan en daarom verwijderd worden, moeten ook gemarkeerd worden om die reden (reason for
#removal: too close to hospital). Dus pas op, je moet dat ook nog apart doen, als je enkel de harstilstanden verwijdert en niet ook de AED's,
#dan lijkt het alsof de AED's die te dicht stonden ook verwijderd werden omdat ze geen harstilstand coverden, terwijl dat eignelijk niet de reden was.

#OPLOSSING: verwijder de harstilstanden die al gered worden door het ziekenhuis niet, maar markeer ze (je kan ze gewoon in een aparte tabel nog 
#eens opslaan OF je maakt een nieuwe kolom in de hartstilstand tabel, "saved_hospital" met waardes yes no. Dan, als een AED enkel harstilstanden dekt
#die al gered waren, dan kan je die AED verwijderen met de reden "person already rescued by hospital" of "too close too hospital to be useful"

In [2]:
#we will use all three instead of just the cardiac arrests, because there are very little instances otherwise. The low amount of
#instances means that the analysis is too much dependent on coincidence. We asume that the locations where the other cardiac problems occur are
#similar to the locations where the cardiac arrests occur.

In [258]:
import pandas as pd
aed_locations = pd.read_csv('aed_locations_coordinates_processed.csv')
card_arrest = pd.read_csv('card_arrest_processed.csv')
hospital_coordinates = pd.read_csv('hospital_coordinates_processed.csv')
removed_AED = pd.read_csv('removed_AED.csv')

In [259]:
#to calculate the distances, we use haversine formula, where we take the shortest distance between two points (a straight line).
#this is not always realistic (suppose an AED is placed on the corner of a building, then a straight line is impossible to take). Therefore we
#add 15% distance, to account for this.

In [260]:
aed_locations = aed_locations.rename(columns={'id': 'aed_id'})
aed_locations

,aed_id,address,number,municipality,latitude,longitude,postal_code,province
0,71.0,Place Saint - Lambert,0,Liège,50.750721,5.933967,4020.0,Liège
1,72.0,Rue du Doyard,0,Lierneux,50.285043,5.785950,4990.0,Liège
2,73.0,Fond Saint Servais,0,Liège,50.646765,5.570987,4000.0,Liège
3,74.0,Rue des Prémontrès,12,Liège,50.638047,5.574046,4020.0,Liège
4,76.0,Rue du Parc,1,Jehay,50.577233,5.323038,4540.0,Liège
...,...,...,...,...,...,...,...,...
11644,16654.0,Farnières,4,Grand Halleux,50.310813,5.880418,6698.0,Liège
11645,16660.0,Chaussée de Marche,799,Wierde,50.426430,4.929522,5100.0,Namur
11646,16661.0,Nekkerspoel-Borcht,19,Mechelen,51.025217,4.504148,2800.0,Antwerpen
11647,16664.0,Nieuwe Dreef,17,Lokeren,51.097802,3.911744,9160.0,Oost-Vlaanderen


In [261]:
card_arrest_all = card_arrest
#eigenlijk kan je gewoon de tijden nemen, de tijden tot interventie, en dan weet je al of de persoon gered is door een ziekenhuis. Je hoeft dan niet
#meer de afstand tov een ziekenhuis te berekenen, want zo ben je er al. Het probleem is dat er veel NA's zijn.

In [262]:
card_arrest_list = card_arrest['eventType_trip'].unique().tolist()
card_arrest_list

['P011 - Chest pain',
 'P039 - Cardiac problem (other than thoracic pain)',
 'P003 - Cardiac arrest',
 'P008 - Patient with defibrillator - pacemaker',
 'P011 N05 - PIJN OP DE BORST',
 'P011 N01 - PIJN OP DE BORST',
 'P039 N05 - CARDIAAL PROBLEEM (ANDERE DAN PIJN AAN DE BORST)',
 'P039 N03 - CARDIAAL PROBLEEM (ANDERE DAN PIJN AAN DE BORST)',
 'P011 N03 - PIJN OP DE BORST',
 'P039 N01 - CARDIAAL PROBLEEM (ANDERE DAN PIJN AAN DE BORST)',
 'P011 N04 - PIJN OP DE BORST',
 'P003  N01 - HARTSTILSTAND - DOOD - OVERLEDEN',
 'P039 N04 - CARDIAAL PROBLEEM (ANDERE DAN PIJN AAN DE BORST)',
 'P008 N03 - PATIËNT MET DEFIBRILLATOR OF PACEMAKER',
 'P008 N01 - PATIËNT MET DEFIBRILLATOR OF PACEMAKER',
 'P011 N02 - PIJN OP DE BORST',
 'P003  N05 - HARTSTILSTAND - DOOD - OVERLEDEN',
 'P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST)',
 'P011 - PIJN OP DE BORST',
 'P003 - HARTSTILSTAND - DOOD - OVERLEDEN',
 'P008 - PATIËNT MET DEFIBRILLATOR OF PACEMAKER']

In [263]:
#in a real world situation, we would if an AED would be useful for all three types of emergencies (P003, P011, P039), or
#only for the cardiac arrests (P003). We will asume that it is only useful for the cardiac arrests, thus we drop the other instances
card_arrest_limited = card_arrest[card_arrest['eventType_trip'].str.contains("P003")]
card_arrest_limited

,eventType_trip,latitude_intervention,longitude_intervention,parsed_T0,parsed_T3,time_difference_seconds,province
9,P003 - Cardiac arrest,50.838080,4.304840,2022-09-06 20:31:42.666252,NaN,NaN,BRU
13,P003 - Cardiac arrest,50.824800,4.381850,2022-09-07 02:36:15.829707,2022-09-07 00:42:25.655123,-6830.174584,BRU
16,P003 - Cardiac arrest,50.838480,4.444410,2022-09-07 12:32:13.510586,2022-09-07 10:49:43.653950,-6149.856636,BRU
33,P003 - Cardiac arrest,50.849480,4.320340,2022-09-08 13:40:50.167814,2022-09-08 11:52:58.220448,-6471.947366,BRU
58,P003 - Cardiac arrest,50.845630,4.308210,2022-09-10 01:14:34.606094,2022-09-10 00:22:34.260161,-3120.345933,BRU
...,...,...,...,...,...,...,...
56248,P003 - HARTSTILSTAND - DOOD - OVERLEDEN,50.937815,4.132712,2023-05-29 23:04:12,NaN,NaN,OVL
56250,P003 - HARTSTILSTAND - DOOD - OVERLEDEN,51.009981,3.899596,2023-05-30 07:46:19,2023-05-30 08:09:56,1417.000000,OVL
56262,P003 - HARTSTILSTAND - DOOD - OVERLEDEN,50.841218,3.902211,2023-05-30 19:05:35,2023-05-30 19:19:42,847.000000,OVL
56270,P003 - HARTSTILSTAND - DOOD - OVERLEDEN,51.127259,3.676424,2023-05-31 07:04:47,2023-05-31 07:21:31,1004.000000,OVL


In [139]:
#we test if the cardiac arrest and the other cardiac problems match in distribution, to see if we can use the bigger dataset
#instead of the smaller, which would help combat randomness resulting in variance in the solution.
#the plot shows a very good overlap, justifying the use of the larger dataset, instead of the smaller one.
import folium
# approximately the center of Belgium coordinates
belgium_center_lat = 50.8503
belgium_center_lon = 4.3517

m = folium.Map(location=[belgium_center_lat, belgium_center_lon], zoom_start=8)

# add all cardiac related intervention locations to the map
for _, row in card_arrest_all.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip='Any Cardiac Problem'
    ).add_to(m)

# add only the cardiac arrests
for _, row in card_arrest_limited.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip='Specifically Cardiac Arrest'
    ).add_to(m)

# save the map to an HTML file, in order to inspect the results. The results seem to indicate that the pattern in occurences is very similar for 
#the different cardiac interventions. Hence we can use all the data.
m.save('cardiac_specific_general.html')

Timings of intervention

In [264]:
#a substantial amount of cardiac arrests does not contain the time needed for intervention.
na_count = card_arrest_all['time_difference_seconds'].isna().sum()
print(f"Number of NA values in 'time_difference_seconds': {na_count}")

Number of NA values in 'time_difference_seconds': 5902


In [265]:
#je kan de tijd nodig om een bepaalde afstand te bereiken berekenen door gebruik te maken van de data. Bereken de afstand
#tussen hospital en startlocatie -> maar startlocatie ontbreekt? Startlocatie is wel aanwezig, maar dan moet dat terug aangepast worden in de 
#preprocessing. en dat is wel echt aids. kan gewoon gemiddelde waarde nemen van tijd. (kijk ook naar de minimum waarde in tijd).

In [266]:
#the time needed to reach a cardiac arrest are already available in the time_difference_seconds variable

In [267]:
#there are negative values present in the timings column. Even if asuming we can simply convert them to positive values, the 
#values are unrealistically high for practically all (more than an hour), so we consider these values as missing.

negative_values = card_arrest[card_arrest_all['time_difference_seconds'] < 0]['time_difference_seconds']

negative_count = negative_values.count()

negative_values_list = negative_values.tolist()

print(negative_values_list)
print(negative_count)

percentile_99 = card_arrest_all['time_difference_seconds'].quantile(0.99)
percentile_1 = card_arrest_all['time_difference_seconds'].quantile(0.01)
print(percentile_99)
print(percentile_1)

[-3828.336006, -6329.451913, -6374.980376, -6226.958482, -6723.857596, -6622.271904, -6272.545074, -6645.216087, -6830.174584, -6429.657075, -6471.686128, -6149.856636, -6379.23241, -5927.773827, -6539.25461, -5223.564681, -6119.501612, -6251.279365, -6682.99007, -6418.806807, -6678.707167, -6489.133732, -6228.706706, -6257.264308, -6471.947366, -5911.326682, -6828.93195, -6316.898505, -6670.486293, -6133.761079, -6258.396155, -5565.02746, -6608.818931, -6486.533399, -4938.405405, -6433.903585, -6268.497188, -5986.855117, -6551.280324, -6549.674492, -3120.345933, -6677.050234, -6294.936732, -6376.730656, -6512.320656, -6596.656984, -6760.889608, -6578.255646, -5111.783329, -6335.820859, -6583.052696, -5751.110476, -6675.237942, -6424.174979, -6530.920975, -6727.28127, -6588.99302, -6642.967015, -6341.608573, -6732.075481, -5880.029328, -6308.109963, -6583.277528, -6403.207927, -6489.451659, -6189.837555, -6379.216647, -6349.068187, -6496.475852, -6386.091593, -6661.333142, -2446.532987

In [268]:
#convert negative values to missing values (NaN), since these values are not realistic and should not be used any further.
card_arrest_all.loc[card_arrest['time_difference_seconds'] < 0, 'time_difference_seconds'] = np.nan

#considering the average time needed of 14 minutes and 58 seconds and the median value of 12 minutes 26 seconds, we consider the missing values as not
#reached in time by the ambulance.

min_value = card_arrest['time_difference_seconds'].min()
max_value = card_arrest['time_difference_seconds'].max()
median_value = card_arrest['time_difference_seconds'].median()
mean_value = card_arrest['time_difference_seconds'].mean()

# Print the results
print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")
print(f"Median value: {median_value}")
print(f"Average value: {mean_value}")


Minimum value: 32.093
Maximum value: 4820474.0
Median value: 804.891
Average value: 157863.65753508548


In [269]:
missing_times = card_arrest_all['time_difference_seconds'].isnull().sum()
missing_times

np.int64(9847)

In [270]:
#it is hard to find the exact time a person with cardiac arrest can survive without receiving aid. This exact value influences 
#the results of the analysis drastically however. In a real analysis the stakeholder would be contacted to provide an estimate of this time.
#we will asume that a person can go for 3 minutes without receiving aid.

#we remove the cardiac arrests that are reached within 3 minutes, since these do not need an AED in order to be helped. We will use the remaining
#cardiac arrest for the rest of the analysis.

In [271]:
#create table that contains all the cardiac arrests that got reached within 3 minutes. We also create an id variable, to be used
#later on.
card_arrest_all['card_id'] = card_arrest_all.index

In [272]:
#only 98 could be reached in time
card_arrest_reached = card_arrest_all[card_arrest_all['time_difference_seconds'] < 180] 
card_arrest_reached

,eventType_trip,latitude_intervention,longitude_intervention,parsed_T0,parsed_T3,time_difference_seconds,province,card_id
1735,P039 - Cardiac problem (other than thoracic pain),50.813070,4.381850,2022-12-07 23:13:46.949822,2022-12-07 23:15:38.325266,111.375444,BRU,1735
5275,P011 N01 - PIJN OP DE BORST,50.853468,4.346246,2022-06-15 15:50:20.000000,2022-06-15 15:52:37.000000,137.000000,BRU,5275
6958,P011 - Chest pain,51.257080,4.415370,2022-06-16 08:49:31.000000,2022-06-16 08:51:53.601000,142.601000,ANT,6958
6973,P003 - Cardiac arrest,51.228920,5.077190,2022-06-17 02:04:21.000000,2022-06-17 02:05:54.387000,93.387000,ANT,6973
6988,P011 - Chest pain,51.271110,5.084000,2022-06-17 17:26:46.000000,2022-06-17 17:28:13.035000,87.035000,ANT,6988
...,...,...,...,...,...,...,...,...
54317,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.930276,3.644738,2023-02-27 16:09:17,2023-02-27 16:10:57,100.000000,OVL,54317
54593,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.930276,3.644738,2023-03-11 19:58:34,2023-03-11 20:00:35,121.000000,OVL,54593
55148,P011 - PIJN OP DE BORST,51.101943,4.000068,2023-04-05 14:12:26,2023-04-05 14:14:27,121.000000,OVL,55148
55194,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),51.101943,4.000068,2023-04-07 11:48:36,2023-04-07 11:50:03,87.000000,OVL,55194


In [235]:
#dan kan je op deze remaining AED's bootstrap toepassen. Dan netwerk maken, van de AED's. hoeft geen echt netwerk te zijn, maar
#in ieder geval mag 1 hartstilstand maar uiteindelijk verbonden zijn met 1 AED, maar 1 AED kan wel verbonden zijn met meerdere hartstilstanden.

In [236]:
#people can run at about 10 km per hour, so the distance ran in 3 minutes would be 500 meter. Since it's unlikely that you can run in a straight line
#from occurrence of cardiac arrest to AED, we add another 15% in time needed to reach the AED. This translates to to the AED needing to be within 
#approximately 435 meter. Since the person would also need to run back and forth, the distance needs to be 217.5 meter or less.


In [273]:
import numpy as np
from scipy.spatial import cKDTree
from geopy.distance import great_circle

# haversine function to calculate the distance between two points on Earth's surface
def haversine(coord1, coord2):
    return great_circle(coord1, coord2).meters  # use meters or kilometers as needed

# extract coordinates and IDs from the tables
arrest_coords = np.array(list(zip(card_arrest_all['latitude_intervention'], card_arrest_all['longitude_intervention'])))
arrest_ids = card_arrest_all['card_id'].values

aed_coords = np.array(list(zip(aed_locations['latitude'], aed_locations['longitude'])))
aed_ids = aed_locations['aed_id'].values

# build a KDTree for AED locations
aed_tree = cKDTree(aed_coords)

# distance threshold X (e.g., in meters)
X = 217.5

# initialize an empty list to store the table entries
table_entries = []

# populate the table
for arrest_index, arrest_coord in enumerate(arrest_coords):
    arrest_id = arrest_ids[arrest_index]
    
    # find all AEDs within distance X
    aeds_within_X = aed_tree.query_ball_point(arrest_coord, X / 6371000)
    
    for aed_index in aeds_within_X:
        aed_coord = aed_coords[aed_index]
        distance = haversine(arrest_coord, aed_coord)
        aed_id = aed_ids[aed_index]
        table_entries.append((distance, arrest_id, aed_id))

# convert the table into a DataFrame for easier manipulation
columns = ['distance', 'card_id', 'aed_id']
table_df = pd.DataFrame(table_entries, columns=columns)

# filter the table to keep only the closest AED for each cardiac arrest
filtered_table_df = table_df.loc[table_df.groupby('card_id')['distance'].idxmin()]


# remove the AED's connected to a cardiac arrest that already got reached in time by a professional
filtered_table_df_cleaned = filtered_table_df[~filtered_table_df['card_id'].isin(card_arrest_reached['card_id'])]

# add the removed AED's that were already reached by a professional to the removed_AED table, with their corresponding reason
rows_to_remove = filtered_table_df[filtered_table_df['card_id'].isin(card_arrest_reached['card_id'])]

aed_rows_to_remove = aed_locations[aed_locations['aed_id'].isin(rows_to_remove['aed_id'])] #als ik enkel de index van de AED's wou is dit eigenlijk
#overbodig.
aed_rows_to_remove['reason'] = 'already_reached'

removed_AED = pd.concat([removed_AED, aed_rows_to_remove[['aed_id', 'reason']].rename(columns={'aed_id': 'id'})], ignore_index=True)



# the remaining AEDs in the filtered table are valid. The rest is considered underperforming
valid_aed_ids = filtered_table_df_cleaned['aed_id'].unique()


valid_aeds = aed_locations[aed_locations['aed_id'].isin(valid_aed_ids)]

In [274]:
#only 25 AED's are well "performing" well, out of the approximately 11 000 valid AED's tested in this step. That's around 0.2% 
#of the tested AED's.
len(valid_aeds)

25

In [275]:
removed_AED

,id,reason,province,municipality
0,7323.0,address_missing,Limburg,Neerpelt
1,7467.0,address_missing,Luxembourg,Saint-Hubert
2,9010899.0,address_missing,NaN,NaN
3,9010900.0,address_missing,NaN,NaN
4,1412.0,not_available,Hainaut,La Hestre
...,...,...,...,...
3573,16633.0,missing_coordinates,Liège,Flémalle
3574,14575.0,missing_coordinates,Luxembourg,Arlon
3575,16658.0,missing_coordinates,Hainaut,Pecq
3576,16659.0,missing_coordinates,Hainaut,Pecq


In [276]:
#de AED's die verwijderd zijn nu nog toevoegen aan removed_AED
#vergelijken tussen de valid AED's en de AED's van ervoor. ook oppassen dat er geen duplicates in komen, simpelweg door duplicates te verwijderen,
#waarbij je "voorrang" geeft aan instances waar de reden "already_reached" is.
#aed_locations
#valid_aeds
removed_AED

,id,reason,province,municipality
0,7323.0,address_missing,Limburg,Neerpelt
1,7467.0,address_missing,Luxembourg,Saint-Hubert
2,9010899.0,address_missing,NaN,NaN
3,9010900.0,address_missing,NaN,NaN
4,1412.0,not_available,Hainaut,La Hestre
...,...,...,...,...
3573,16633.0,missing_coordinates,Liège,Flémalle
3574,14575.0,missing_coordinates,Luxembourg,Arlon
3575,16658.0,missing_coordinates,Hainaut,Pecq
3576,16659.0,missing_coordinates,Hainaut,Pecq


In [277]:
#add the underperforming AED's to the removed_AED data frame

underperforming_aeds = aed_locations[~aed_locations['aed_id'].isin(valid_aeds['aed_id'])]
underperforming_aeds['reason'] = 'underperforming'
underperforming_aeds = underperforming_aeds[['aed_id', 'reason', 'province', 'municipality']]
underperforming_aeds = underperforming_aeds.rename(columns = {'aed_id': 'id'})

removed_AED = pd.concat([removed_AED, underperforming_aeds], ignore_index=True)
removed_AED

C:\Users\eloua\AppData\Local\Temp\ipykernel_20008\3796858515.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  underperforming_aeds['reason'] = 'underperforming'


,id,reason,province,municipality
0,7323.0,address_missing,Limburg,Neerpelt
1,7467.0,address_missing,Luxembourg,Saint-Hubert
2,9010899.0,address_missing,NaN,NaN
3,9010900.0,address_missing,NaN,NaN
4,1412.0,not_available,Hainaut,La Hestre
...,...,...,...,...
15197,16654.0,underperforming,Liège,Grand Halleux
15198,16660.0,underperforming,Namur,Wierde
15199,16661.0,underperforming,Antwerpen,Mechelen
15200,16664.0,underperforming,Oost-Vlaanderen,Lokeren


In [278]:
#now, in the case that there are AED which cover a cardiac arrest that is already reached by a professional, it will be so that
#this instance will be present twice in the removed_AED data frame, once with 'reason' value 'underperforming', once with 'reason' value
#'already_reached'. We will remove the instance for which the value is 'underperforming', since then the correct reason for removal remains.

# sort the DataFrame so that 'already_reached' comes before 'underperforming'
removed_AED = removed_AED.sort_values(by=['id', 'reason'], ascending=[True, True])

# drop duplicates, keeping the first occurrence (which will be 'already_reached')
removed_AED_cleaned = removed_AED.drop_duplicates(subset=['id', 'province', 'municipality'], keep='first')
removed_AED_cleaned
#there was no overlap for X = 217.5 so nothing changed.

,id,reason,province,municipality
753,13.0,missing_coordinates,Hainaut,SOIGNIES
754,70.0,missing_coordinates,Liège,Ayeneux
3578,71.0,underperforming,Liège,Liège
3579,72.0,underperforming,Liège,Lierneux
3580,73.0,underperforming,Liège,Liège
...,...,...,...,...
14406,9015882.0,underperforming,Oost-Vlaanderen,OOSTAKKER
14407,9015883.0,underperforming,Antwerpen,KAPELLEN
14408,9015884.0,underperforming,Antwerpen,HERENTHOUT
14409,9015885.0,underperforming,Antwerpen,HOOGSTRATEN


creating synthetic instances to test our results with

In [279]:
#KDE sampling and poisson based sampling were also used to create synthetic instances, however the results were very unrealistic,
#based on a plotting of synthetic instances and original instances.
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

#perturbation of 0.01 means the synthetic instance occurs about a couple streets further, which is what we want
#we take a very large amount of clusters, in order to avoid just creating synthetic instances that match our future clustering analysis and thus 
#providing us with artificially high performance metrics, simply because it matches the used analysis method. A high amount of clusters also matches
#the the observed data well
np.random.seed(11)
def combined_sampling(df, num_samples, perturbation_std=0.01):
    coords = df[['latitude_intervention', 'longitude_intervention']].values
    kmeans = KMeans(n_clusters=1300).fit(coords)
    
    # perturbation around cluster centers
    new_data = []
    for center in kmeans.cluster_centers_:
        for _ in range(num_samples // len(kmeans.cluster_centers_)):
            lat_noise = np.random.normal(0, perturbation_std)
            lon_noise = np.random.normal(0, perturbation_std)
            new_data.append({
                'latitude_intervention': center[0] + lat_noise,
                'longitude_intervention': center[1] + lon_noise
            })
    
    return pd.DataFrame(new_data)

df_synthetic = combined_sampling(card_arrest, num_samples=56000, perturbation_std=0.01)


In [ ]:
#the plot shows that the synthetic data created using this method matches the observed data well. However, we will
#still teste whether or not the synthetic data is good by comparing the performance of the AED's on this synthetic dataset to
#the performance on the real dataset.
#This data can be used later, in order to test the performance of our newly located AED's.
m = folium.Map(location=[belgium_center_lat, belgium_center_lon], zoom_start=8)

# add original cardiac arrest locations to the map
for _, row in card_arrest.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip='Original Cardiac Intervention'
    ).add_to(m)

# add synthetic cardiac arrest locations to the map
for _, row in df_synthetic.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='orange',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip='Synthetic Cardiac Intervention'
    ).add_to(m)

# save the map to an HTML file, in order to inspect the results. The results seem realistic.
m.save('cardiac_arrests_map_belgium_clustering.html')

In [187]:
#using this to see how much the coordinates should be ofset using a uniform distribution
#0.0015 is een straat verder, wat we on average willen hebben, dus [-0.0030, 0.0030] is de range die we willen hebben
testset = pd.DataFrame({
    'latitude': [50.916854, 50.918354, 50.919854, 50.915354, 50.921354], #increments of 0.0015
    'longitude': [4.307399, 4.308899, 4.310399, 4.305899, 4.321899]
})



m = folium.Map(location=[belgium_center_lat, belgium_center_lon], zoom_start=8)

# add original cardiac arrest locations to the map
for _, row in testset.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip='Original Cardiac Intervention'
    ).add_to(m)



# save the map to an HTML file, in order to inspect the results.
m.save('coord_testing.html')

In [280]:
#cardiac arrests randomly moved by about a street [-0.003, 0.003] in longitude and latitude. By doing this, we essentially
#maintain the same structure of the original data (density, distribution). We only change the exact locations of occurrence, for example a cardiac
#arrest occurring a street further than it did in the original data. This is to make sure our solution doesn't overfit on the original data.
synthetic_uniform = card_arrest_all.copy()
np.random.seed(49)

#generate random values to add/subtract from both latitude and longitude
random_offsets = np.random.uniform(-0.003, 0.003, size=(len(card_arrest_all), 2))

#apply the offsets to the latitude and longitude
synthetic_uniform['latitude_intervention'] += random_offsets[:, 0]
synthetic_uniform['longitude_intervention'] += random_offsets[:, 1]
synthetic_uniform
#50.877980
#4.392290

,eventType_trip,latitude_intervention,longitude_intervention,parsed_T0,parsed_T3,time_difference_seconds,province,card_id
0,P011 - Chest pain,50.876786,4.390772,2022-09-06 14:51:41.968677,NaN,NaN,BRU,0
1,P011 - Chest pain,50.898338,4.360520,2022-09-06 15:08:52.857352,2022-09-06 14:05:04.521346,NaN,BRU,1
2,P011 - Chest pain,50.857020,4.426321,2022-09-06 16:36:15.795918,2022-09-06 14:50:46.344005,NaN,BRU,2
3,P011 - Chest pain,50.809662,4.345693,2022-09-06 17:14:30.554547,2022-09-06 15:28:15.574171,NaN,BRU,3
4,P039 - Cardiac problem (other than thoracic pain),50.871059,4.363177,2022-09-06 18:21:26.211055,NaN,NaN,BRU,4
...,...,...,...,...,...,...,...,...
56279,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.980519,3.985596,2023-05-31 19:47:06,2023-05-31 19:56:44,578.0,OVL,56279
56280,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),51.128831,4.111268,2023-05-31 20:41:28,2023-05-31 20:53:46,738.0,OVL,56280
56281,P003 - HARTSTILSTAND - DOOD - OVERLEDEN,50.893237,3.854881,2023-05-31 20:45:48,2023-05-31 20:57:17,689.0,OVL,56281
56282,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.963299,3.756292,2023-05-31 21:38:03,2023-05-31 21:54:16,973.0,OVL,56282


In [205]:
#comparing the original instances to the synthetic instances generated using the uniform distribution
m = folium.Map(location=[belgium_center_lat, belgium_center_lon], zoom_start=8)

# add original cardiac arrest locations to the map
for _, row in card_arrest_all.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip='Original Cardiac Intervention'
    ).add_to(m)

# add synthetic cardiac arrest locations to the map
for _, row in synthetic_uniform.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='orange',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip='Synthetic Cardiac Intervention'
    ).add_to(m)

# save the map to an HTML file, in order to inspect the results. The results seem realistic.
m.save('cardiac_arrests_map_belgium_uniform.html')

In [102]:
#the plot shows that the synthetic data created using this method matches the observed data well. However, we will
#still teste whether or not the synthetic data is good by comparing the performance of the AED's on this synthetic dataset to
#the performance on the real dataset.
#This data can be used later, in order to test the performance of our newly located AED's.
m = folium.Map(location=[belgium_center_lat, belgium_center_lon], zoom_start=8)

# add original cardiac arrest locations to the map
for _, row in card_arrest.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip='Original Cardiac Intervention'
    ).add_to(m)

# add synthetic cardiac arrest locations to the map
for _, row in df_synthetic.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='orange',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip='Synthetic Cardiac Intervention'
    ).add_to(m)

# save the map to an HTML file, in order to inspect the results. The results seem realistic.
m.save('cardiac_arrests_map_belgium_clustering.html')

In [93]:
df_synthetic.to_csv('df_synthetic.csv', index=False)

In [84]:
df_synthetic

,latitude_intervention,longitude_intervention
0,50.918356,4.308899
1,50.915025,4.304220
2,50.897697,4.294722
3,50.917583,4.311041
4,50.913290,4.298429
...,...,...
19495,50.788278,4.283532
19496,50.792272,4.294887
19497,50.767103,4.298737
19498,50.775104,4.303289


In [91]:
#als ik de performance test van de nieuwe locaties, moet ik die ook vergelijken met de originele setup. performance van de 
#originele setup op de synthetic data zou gelijk moeten zijn aan de performance op de originele data, als de bootstrap goed gedaan is.
aed_locations

,address,number,municipality,latitude,longitude,aed_id,postal_code,province
0,Place Saint - Lambert,0,Liège,50.750721,5.933967,71.0,4020.0,Liège
1,Rue du Doyard,0,Lierneux,50.285043,5.785950,72.0,4990.0,Liège
2,Fond Saint Servais,0,Liège,50.646765,5.570987,73.0,4000.0,Liège
3,Rue des Prémontrès,12,Liège,50.638047,5.574046,74.0,4020.0,Liège
4,Rue du Parc,1,Jehay,50.577233,5.323038,76.0,4540.0,Liège
...,...,...,...,...,...,...,...,...
11646,Farnières,4,Grand Halleux,50.310813,5.880418,16154.0,6698.0,Liège
11647,Chaussée de Marche,799,Wierde,50.426430,4.929522,16157.0,5100.0,Namur
11648,Nekkerspoel-Borcht,19,Mechelen,51.025217,4.504148,16158.0,2800.0,Antwerpen
11649,Nieuwe Dreef,17,Lokeren,51.097802,3.911744,16159.0,9160.0,Oost-Vlaanderen


In [281]:
#already test the performance of the original setup on the bootstrap, as a way to verify whether the bootstrap is realistic. If
#it is, then the resulting performance (i.e. the number of effective AED's) should be similar.
df_synthetic

#recycling the code used to get the valid AED's on the real dataset

# extract coordinates and IDs from the tables
synth_coords = np.array(list(zip(df_synthetic['latitude_intervention'], df_synthetic['longitude_intervention'])))
synth_ids = df_synthetic.index



aed_coords = np.array(list(zip(aed_locations['latitude'], aed_locations['longitude'])))
aed_ids = aed_locations['aed_id'].values

# build a KDTree for AED locations
aed_tree = cKDTree(aed_coords)

# distance threshold X (e.g., in meters)
X = 217.5

# initialize an empty list to store the table entries
table_entries = []

# populate the table
for arrest_index, arrest_coord in enumerate(synth_coords):
    arrest_id = synth_ids[arrest_index]
    
    # find all AEDs within distance X
    aeds_within_X = aed_tree.query_ball_point(arrest_coord, X / 6371000)
    for aed_index in aeds_within_X:
        aed_coord = aed_coords[aed_index]
        distance = haversine(arrest_coord, aed_coord)
        aed_id = aed_ids[aed_index]
        table_entries.append((distance, arrest_id, aed_id))

# convert the table into a DataFrame for easier manipulation
columns = ['distance', 'card_id', 'aed_id']
table_df = pd.DataFrame(table_entries, columns=columns)

# filter the table to keep only the closest AED for each cardiac arrest
filtered_table_df = table_df.loc[table_df.groupby('card_id')['distance'].idxmin()]

#no cardiac arrests could be reached here, but the amount of overlap is very small anyway, so it won't really affect the results


# the remaining AEDs in the filtered table are valid. The rest is considered underperforming
valid_aed_ids = filtered_table_df['aed_id'].unique()


valid_aeds_synthetic = aed_locations[aed_locations['aed_id'].isin(valid_aed_ids)]


In [282]:
valid_aeds_synthetic #warempel, de performance is toch gewoon veel slechter op die synthetic instances en de code werkt. (dit is 
#met hetzelfde aantal als originele set, dus 56 000.
#andere bootstrap vinden dan
#de huidige bootstrap lijkt niet echt goed om te kunnen gaan met verschillende densities aanwezig in de data, bv. brussel krijgt relatief weinig
#cardiac arrests en random gebied relatief veel, terwijl brussel veel meer zou moeten krijgen dan de rest.

#OPLOSSING: geen fancy bootstrap gebruiken, maar letterlijk gewoon in een range van +0.00005 -0.00005 uniform voor elke coordinate optellen/verminderen
#, en dan heb je essentieel dezelfde dataset, maar dan alsof alles random een straat verder of dichter gebeurt.

,aed_id,address,number,municipality,latitude,longitude,postal_code,province
2184,3726.0,Kruidtuinlaan,55,Brussel,50.852967,4.365200,1000.0,Bruxelles-Brussel
4050,6189.0,Rue Sylvain Guyaux,49,La,50.475697,4.183766,7100.0,Hainaut
4713,7150.0,AVENUE MOZART),84,Bruxelles,50.810714,4.336772,1190.0,Bruxelles-Brussel
6076,9157.0,Rue Emile Bouilliot,22,Bruxelles,50.818973,4.354166,1050.0,Bruxelles-Brussel
6088,9174.0,Rue de Verviers,89,Eupen,50.631614,6.026232,4700.0,Liège
8584,103012.0,avenue de la Métrologie,0,Bruxelles,50.894029,4.427928,1130.0,Bruxelles-Brussel


In [283]:
#now we test on the synthetic instances generated using a uniform distribution

#recycling the code used to get the valid AED's on the real dataset

# extract coordinates and IDs from the tables
synth_coords = np.array(list(zip(synthetic_uniform['latitude_intervention'], synthetic_uniform['longitude_intervention'])))
synth_ids = synthetic_uniform.index



aed_coords = np.array(list(zip(aed_locations['latitude'], aed_locations['longitude'])))
aed_ids = aed_locations['aed_id'].values

# build a KDTree for AED locations
aed_tree = cKDTree(aed_coords)

# distance threshold X (e.g., in meters)
X = 217.5

# initialize an empty list to store the table entries
table_entries = []

# populate the table
for arrest_index, arrest_coord in enumerate(synth_coords):
    arrest_id = synth_ids[arrest_index]
    
    # find all AEDs within distance X
    aeds_within_X = aed_tree.query_ball_point(arrest_coord, X / 6371000)
    for aed_index in aeds_within_X:
        aed_coord = aed_coords[aed_index]
        distance = haversine(arrest_coord, aed_coord)
        aed_id = aed_ids[aed_index]
        table_entries.append((distance, arrest_id, aed_id))

# convert the table into a DataFrame for easier manipulation
columns = ['distance', 'card_id', 'aed_id']
table_df = pd.DataFrame(table_entries, columns=columns)

# filter the table to keep only the closest AED for each cardiac arrest
filtered_table_df = table_df.loc[table_df.groupby('card_id')['distance'].idxmin()]

#no cardiac arrests could be reached here, but the amount of overlap is very small anyway, so it won't really affect the results


# the remaining AEDs in the filtered table are valid. The rest is considered underperforming
valid_aed_ids = filtered_table_df['aed_id'].unique()


valid_aeds_synthetic_uniform = aed_locations[aed_locations['aed_id'].isin(valid_aed_ids)]


In [284]:
#9 AED's are valid using the uniform distribution, which comes closer to the 25 of the real dataset. The difference between 
#the amount that is valid in the real dataset and the amount that is valid in the uniform synthetic dataset can be caused by the fact
#that this method does not account for the fact that cardiac arrests usually occur somewhere along a street, and AED's also are placed somewhere
#along a street; the synthetic instances are randomly moved, so they occur also at random locations (e.g. in an open field, not along a street).
#the most realistic synthetic instances would likely be created by randomly varying the street number. However, since not all streets have the same
#amount of numbers, we cannot use this.
valid_aeds_synthetic_uniform

,aed_id,address,number,municipality,latitude,longitude,postal_code,province
721,1566.0,Rue Haute,0,Bruxelles,50.841854,4.352098,1000.0,Bruxelles-Brussel
1238,2379.0,De Smet de Naeyerlaan,145,Brussel,50.874738,4.328364,1090.0,Bruxelles-Brussel
1574,2882.0,Westendelaan,24,Middelkerke,51.184783,2.819490,8430.0,West-Vlaanderen
2226,3793.0,Allée Max Quintart,1,Bon-Secours,50.499157,3.612549,7603.0,Hainaut
2385,4013.0,Pl du Roi Baudouin,1,Bruxelles,50.863186,4.296461,1082.0,Bruxelles-Brussel
5180,7815.0,Rue de Trèves,5,Bruxelles,50.839714,4.373924,1050.0,Bruxelles-Brussel
5564,8346.0,Merantistraat,1,Antwerpen,51.239861,4.411151,2030.0,Antwerpen
9438,14480.0,rue Picard,11,Bruxelles,50.865415,4.344825,1000.0,Bruxelles-Brussel
9941,15178.0,Avenue de la Gare,39,Aubange,49.566613,5.802451,6790.0,Luxembourg


In [ ]:
#als het moet kan ik die bootstrap shit laten vallen. Eerst focussen op de rest verder doen, dus de optimale plaatsing van de
#AED's vinden. Ook al aan pipeline denken daarvoor. Gewoon veel verschillende methodes proberen. Doe het op de echte data, gebruik het aantal 
#geredde mensen met de AED's als criterium van performance.

#PAS OP: de card arrests van de mensen die al gered zijn moeten voor deze training dataset ook er terug uitgehaald worden, want je wil niet een 
#AED plaatsen in de buurt daarvan.

Finding the new locations

In [ ]:
#methode nodig die rekening kan houden met de verschillende densities. want eigenlijk gaat het primair daarover, gewoon de 
#AED's plaatsen op de locaties waar cardiac arrests zeer dense zijn.

#kan ook kijken naar hoe de AED's momenteel geplaatst zijn, om te zien of er een patroon is. ook kijken naar de AED's die goed zijn, waar die staan.

In [173]:
valid_aeds

,aed_id,address,number,municipality,latitude,longitude,postal_code,province
214,777.0,Sporthalstraat,12,Tienen,50.818920,4.945635,3300.0,Vlaams-Brabant
1583,2892.0,Vooruitgangstraat,80,Brussel,50.860697,4.359930,1030.0,Bruxelles-Brussel
2003,3463.0,Rue Neuve,79,Dison,50.605078,5.857454,4820.0,Liège
2032,3500.0,Rue de la Vellerie,2,Mouscron,50.729826,3.209694,7700.0,Hainaut
2722,4438.0,Voie d Hainin,27,Boussu,50.433790,3.791288,7300.0,Hainaut
2801,4540.0,Rue de Bruxelles,0,Fleurus,50.485290,4.551276,6220.0,Hainaut
3090,4935.0,Avenue de l Espérance,1,Fleurus,50.452021,4.535602,6220.0,Hainaut
3362,5304.0,Damse Vaart Oost,1,Damme,51.266547,3.298095,8340.0,West-Vlaanderen
3782,5842.0,Hopland,0,Antwerpen,51.216721,4.411910,2000.0,Antwerpen
4117,6270.0,Mellestraat,0,Heule,50.846227,3.231480,8501.0,West-Vlaanderen


In [175]:
#plot to see where the valid AED's are located, perhaps we can learn something from this

# approximately the center of Belgium coordinates
belgium_center_lat = 50.8503
belgium_center_lon = 4.3517

m = folium.Map(location=[belgium_center_lat, belgium_center_lon], zoom_start=8)

# add cardiac arrests to the map
for _, row in card_arrest.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.2,
        tooltip='Cardiac Arrest'
    ).add_to(m)

# add AED valid AED's to the map
for _, row in valid_aeds.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.2,
        tooltip='AED'
    ).add_to(m)


m.save('valid_aeds.html')

In [ ]:
#as expected, the valid aed's lie in or close to a city, where the population density is higher. There are also no valid AED's in the less densely
#populated south of Walonia. Therefore, to find the new locations for the AED's, we will need to make use of a method that can make use of the
#density of clusters, since density of the occurrence of cardiac arrests plays a major role in whether or not an AED is useful.

In [ ]:
#clusters vinden
#there is a limited amount of AED's compared to the overall size that should be covered (bereken opervlakte belgie, bereken opervlakte die 
#1 AED dekt, bereken totaal oppervlakte die alle AED's kunnen dekken, bereken oppervlakte van onze herplaatsing AEDs). Identifying locations where
#cardiac arrests occur more often compared to the rest of Belgium and placing AED's there seems like the most valid strategy.
#2 regels:
#-density: for a cluster to be valid, the density needs to be above a certain treshold. This is because density is the most important factor when
#considering to place an AED somewhere, since density decides whether or not the AED will actually be close to the occurrence. It doesn't matter if
#the method finds a perfectly shaped cluster, if all the cardiac arrests are far from each other placing effective AED's will nog be possible.
#-minimum aantal per cluster: in order to avoid finding clusters consisting of cardiac arrests which lie coincidentally close to each other without
#there being a real pattern, a minimum amount of cardiac arrests needs to be in the cluster. This minimum amount should be small, since the chance
#that two or more lie close to each other is small anyway.

In [ ]:
#PAS OP: de card arrests van de mensen die al gered zijn moeten voor deze training dataset ook er terug uitgehaald worden, want je wil niet een
#AED plaatsen in de buurt daarvan.

In [ ]:
#remove the cardiac arrests of people already saved. These are the ones covered by the hospitals, and the ones covered by the
#valid AED's

filtered_table_df

card_arrest_reached

card_arrest_all

In [215]:
card_arrest_all #card_id

,eventType_trip,latitude_intervention,longitude_intervention,parsed_T0,parsed_T3,time_difference_seconds,province,card_id
0,P011 - Chest pain,50.874115,4.390594,2022-09-06 14:51:41.968677,NaN,NaN,BRU,0
1,P011 - Chest pain,50.899509,4.362461,2022-09-06 15:08:52.857352,2022-09-06 14:05:04.521346,NaN,BRU,1
2,P011 - Chest pain,50.854477,4.424170,2022-09-06 16:36:15.795918,2022-09-06 14:50:46.344005,NaN,BRU,2
3,P011 - Chest pain,50.804087,4.347814,2022-09-06 17:14:30.554547,2022-09-06 15:28:15.574171,NaN,BRU,3
4,P039 - Cardiac problem (other than thoracic pain),50.874316,4.366531,2022-09-06 18:21:26.211055,NaN,NaN,BRU,4
...,...,...,...,...,...,...,...,...
56279,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.978256,3.985515,2023-05-31 19:47:06,2023-05-31 19:56:44,578.0,OVL,56279
56280,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),51.127089,4.111784,2023-05-31 20:41:28,2023-05-31 20:53:46,738.0,OVL,56280
56281,P003 - HARTSTILSTAND - DOOD - OVERLEDEN,50.887808,3.856896,2023-05-31 20:45:48,2023-05-31 20:57:17,689.0,OVL,56281
56282,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.965354,3.753030,2023-05-31 21:38:03,2023-05-31 21:54:16,973.0,OVL,56282


In [243]:
filtered_table_df #card_id

,distance,card_id,aed_id
0,2.576553,663,16056.0
1,2.548389,1580,10763.0
3,0.480340,4393,8934.0
6,1.039722,5445,2892.0
7,2.799895,6693,15661.0
8,2.065953,11129,5842.0
9,2.640023,16370,3500.0
11,1.371083,16404,4935.0
12,2.399072,18071,4438.0
13,1.836366,18199,6614.0


In [242]:
card_arrest_reached

,eventType_trip,latitude_intervention,longitude_intervention,parsed_T0,parsed_T3,time_difference_seconds,province,card_id
1735,P039 - Cardiac problem (other than thoracic pain),50.813070,4.381850,2022-12-07 23:13:46.949822,2022-12-07 23:15:38.325266,111.375444,BRU,1735
5275,P011 N01 - PIJN OP DE BORST,50.853468,4.346246,2022-06-15 15:50:20.000000,2022-06-15 15:52:37.000000,137.000000,BRU,5275
6958,P011 - Chest pain,51.257080,4.415370,2022-06-16 08:49:31.000000,2022-06-16 08:51:53.601000,142.601000,ANT,6958
6973,P003 - Cardiac arrest,51.228920,5.077190,2022-06-17 02:04:21.000000,2022-06-17 02:05:54.387000,93.387000,ANT,6973
6988,P011 - Chest pain,51.271110,5.084000,2022-06-17 17:26:46.000000,2022-06-17 17:28:13.035000,87.035000,ANT,6988
...,...,...,...,...,...,...,...,...
54317,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.930276,3.644738,2023-02-27 16:09:17,2023-02-27 16:10:57,100.000000,OVL,54317
54593,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.930276,3.644738,2023-03-11 19:58:34,2023-03-11 20:00:35,121.000000,OVL,54593
55148,P011 - PIJN OP DE BORST,51.101943,4.000068,2023-04-05 14:12:26,2023-04-05 14:14:27,121.000000,OVL,55148
55194,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),51.101943,4.000068,2023-04-07 11:48:36,2023-04-07 11:50:03,87.000000,OVL,55194


In [285]:
#combine the card_id values from filtered_table_df and card_arrest_reached
combined_ids = pd.concat([filtered_table_df['card_id'], card_arrest_reached['card_id']])

filtered_card_arrest_all = card_arrest_all[~card_arrest_all['card_id'].isin(combined_ids)]
filtered_card_arrest_all

,eventType_trip,latitude_intervention,longitude_intervention,parsed_T0,parsed_T3,time_difference_seconds,province,card_id
0,P011 - Chest pain,50.877980,4.392290,2022-09-06 14:51:41.968677,NaN,NaN,BRU,0
1,P011 - Chest pain,50.895780,4.358170,2022-09-06 15:08:52.857352,2022-09-06 14:05:04.521346,NaN,BRU,1
2,P011 - Chest pain,50.855920,4.425920,2022-09-06 16:36:15.795918,2022-09-06 14:50:46.344005,NaN,BRU,2
3,P011 - Chest pain,50.809380,4.347430,2022-09-06 17:14:30.554547,2022-09-06 15:28:15.574171,NaN,BRU,3
4,P039 - Cardiac problem (other than thoracic pain),50.869440,4.360800,2022-09-06 18:21:26.211055,NaN,NaN,BRU,4
...,...,...,...,...,...,...,...,...
56279,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.980111,3.982742,2023-05-31 19:47:06,2023-05-31 19:56:44,578.0,OVL,56279
56280,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),51.131239,4.111499,2023-05-31 20:41:28,2023-05-31 20:53:46,738.0,OVL,56280
56281,P003 - HARTSTILSTAND - DOOD - OVERLEDEN,50.896206,3.856032,2023-05-31 20:45:48,2023-05-31 20:57:17,689.0,OVL,56281
56282,P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),50.965585,3.758418,2023-05-31 21:38:03,2023-05-31 21:54:16,973.0,OVL,56282


In [ ]:
#ook het aantal AED's dat we nu kunnen gebruiken bepalen. is in principe len(removed_AED), maar eigenlijk niet, want
#van sommige weten we niet of ze goed performen, dus zal moeten filteren nog.

In [ ]:
#we will only replace the AED's for which we actually know they are not performing well. We cannot say we need to move an AED
#if we do not know the coordinates, or if the address is missing. We will also move the ones that are not available, although in a real context
#one would have needed to verify why these are not available (because they are for example privately owned).

In [286]:
unique_reasons = removed_AED['reason'].unique()
unique_reasons
remove_reasons = ['underperforming', 'not_available', 'already_reached']

In [287]:
aeds_to_move = removed_AED[removed_AED['reason'].isin(remove_reasons)]
aeds_to_move

,id,reason,province,municipality
3578,71.0,underperforming,Liège,Liège
3579,72.0,underperforming,Liège,Lierneux
3580,73.0,underperforming,Liège,Liège
3581,74.0,underperforming,Liège,Liège
3582,76.0,underperforming,Liège,Jehay
...,...,...,...,...
14405,9015881.0,underperforming,Oost-Vlaanderen,OOSTAKKER
14406,9015882.0,underperforming,Oost-Vlaanderen,OOSTAKKER
14407,9015883.0,underperforming,Antwerpen,KAPELLEN
14408,9015884.0,underperforming,Antwerpen,HERENTHOUT


In [316]:
aed_locations

,aed_id,address,number,municipality,latitude,longitude,postal_code,province
0,71.0,Place Saint - Lambert,0,Liège,50.750721,5.933967,4020.0,Liège
1,72.0,Rue du Doyard,0,Lierneux,50.285043,5.785950,4990.0,Liège
2,73.0,Fond Saint Servais,0,Liège,50.646765,5.570987,4000.0,Liège
3,74.0,Rue des Prémontrès,12,Liège,50.638047,5.574046,4020.0,Liège
4,76.0,Rue du Parc,1,Jehay,50.577233,5.323038,4540.0,Liège
...,...,...,...,...,...,...,...,...
11644,16654.0,Farnières,4,Grand Halleux,50.310813,5.880418,6698.0,Liège
11645,16660.0,Chaussée de Marche,799,Wierde,50.426430,4.929522,5100.0,Namur
11646,16661.0,Nekkerspoel-Borcht,19,Mechelen,51.025217,4.504148,2800.0,Antwerpen
11647,16664.0,Nieuwe Dreef,17,Lokeren,51.097802,3.911744,9160.0,Oost-Vlaanderen


In [318]:
#since aeds_to_move does not contain coordinates, we use it to create a new table that does contain coordinates.

ids_to_move = aeds_to_move['id'].tolist()

aeds_to_move_with_locations = aed_locations[aed_locations['aed_id'].isin(ids_to_move)]
aeds_to_move_with_locations

,aed_id,address,number,municipality,latitude,longitude,postal_code,province
0,71.0,Place Saint - Lambert,0,Liège,50.750721,5.933967,4020.0,Liège
1,72.0,Rue du Doyard,0,Lierneux,50.285043,5.785950,4990.0,Liège
2,73.0,Fond Saint Servais,0,Liège,50.646765,5.570987,4000.0,Liège
3,74.0,Rue des Prémontrès,12,Liège,50.638047,5.574046,4020.0,Liège
4,76.0,Rue du Parc,1,Jehay,50.577233,5.323038,4540.0,Liège
...,...,...,...,...,...,...,...,...
11644,16654.0,Farnières,4,Grand Halleux,50.310813,5.880418,6698.0,Liège
11645,16660.0,Chaussée de Marche,799,Wierde,50.426430,4.929522,5100.0,Namur
11646,16661.0,Nekkerspoel-Borcht,19,Mechelen,51.025217,4.504148,2800.0,Antwerpen
11647,16664.0,Nieuwe Dreef,17,Lokeren,51.097802,3.911744,9160.0,Oost-Vlaanderen


In [ ]:
#nu nog een methode vinden om ze optimaal te plaatsen,
#je kan de originele data en de bootstrap gebruiken als performance metric
#je kan de regels over goede clusters gebruiken
#maar dan nog, hoe bepaal je een goeie exacte locatie? gewoon random droppen in de cluster, of in de vorm van een rooster ofzo? of binnen elke cluster
#een slimme verdeling volgen?

#we willen sowieso boven een bepaalde density raken. dus dat als eerste stap. Dan kan je binnen elke cluster kijken hoe verdelen. bij deze tweede
#stap kan je gebruik maken van performance criterium? Je kan het aantal AED's per cluster toewijzen op basis van het aantal cardiac arrests in de 
#cluster, maar het is niet noodzakelijk zo dat dat ook effectief de beste oplossing is.

#we kunnen sowieso searchen over een aantal waardes voor de params, bv de optimale density.
#meerdere benaderingen (nadat de clusters gevonden zijn):
#-k-means opnieuw doen binnen elke cluster, voor elke AED die toegewezen is aan de cluster (maar dan moeten we al een bepaald aantal toewijzen 
#aan elke cluster, en hoe doe je dat?)
#-Maximal Coverage Location Problem (MCLP): we weten dat elke AED een bepaalde area bedekt. We kunnen de AED's zodanig plaatsen dat maximaal oppervlak
#van de cluster is gedekt. Dat zal lijken alsof we een rooster maken van AED's dan (-> misschien kan je dit combineren met de density binnen de cluster
#zelf nog eens, of met een clustering methode)
#-Incident Frequency Weighted Centroid: If some locations within a cluster have had more incidents than others, you could use a weighted centroid where
#locations with more incidents pull the AED placement closer to them.
#

In [ ]:
#op basis van de resultaten van de dbscan lijkt een methode waar het grootste oppevlak wordt bedekt de beste, binnen in 
#de cluster. de vorm die maximaal bedekt moet worden is dan een cirkel, met als middlepunt de centroid van de cluster, en als straat
#de afstand tot het verste punt. (misschien niet de beste oplossing dit).

In [ ]:
#eerst bepalen welke initiele clustering methode toepassen.

In [288]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


coordinates = filtered_card_arrest_all[['latitude_intervention', 'longitude_intervention']].values


In [301]:
#we want to keep epsilon small, since we are using DBSCAN, we want to avoid making a long, huge cluster, since that would not
#allow us to find effective locations (???????)

#there is no definite answer on how big epsilon should be, i.e. how great the distance between two cardiac arrests should be for them to be 
#considered belonging to the same cluster. Make the distance too big and we end up with useless clusters that contain too many instances too far from
#each other. Too small and we end up with small clusters that are likely going to result in AED locations that are "overfitted".

# Parameters to tune:
epsilon = 0.0018  # Adjust based on your data's scale (distance threshold). 0.0018 is approximately 217.5 meter, moet dit nog op een slimme manier
#doen, X gebruiken die eerder gedefinieerd werd als de afstand waarbinnen een card_arrest mag liggen.
min_samples = 3  # Minimum number of instances in a cluster

#euclidian distance will be accurate enough for our purposes, since the distances are small they are not affected significantly by Earth's curvature
db = DBSCAN(eps=epsilon, min_samples=min_samples, metric='euclidean').fit(coordinates)

# Add cluster labels to your DataFrame
filtered_card_arrest_all['cluster'] = db.labels_
labels = db.labels_


# Create a DataFrame to hold labels and their counts
labels_df = pd.DataFrame(labels, columns=['cluster'])
labels_df['cluster'] = labels_df['cluster'].astype(str)  # Convert to string for easier handling

# Count occurrences of each cluster label
cluster_counts = labels_df['cluster'].value_counts()

# Exclude noise points, which are labeled as '-1'
cluster_counts = cluster_counts[cluster_counts.index != '-1']

# Count how many clusters have each size
size_counts = cluster_counts.value_counts().sort_index()


# -1 indicates noise points that don't belong to any cluster


C:\Users\eloua\AppData\Local\Temp\ipykernel_20008\560306611.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_card_arrest_all['cluster'] = db.labels_


In [295]:
#create a plot to review the solution
import pandas as pd
from sklearn.cluster import DBSCAN
import folium
import numpy as np
import random

# Function to generate a random color
def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))


# Coordinates for Belgium (rough center of the country)
belgium_center = [50.8503, 4.3517]

# Create a Folium map centered on Belgium
m = folium.Map(location=belgium_center, zoom_start=8)

# Generate random colors for each cluster
unique_clusters = set(filtered_card_arrest_all['cluster'])
cluster_colors = {}
for cluster_label in unique_clusters:
    if cluster_label != -1:  # Skip noise
        cluster_colors[cluster_label] = random_color()

#plot original points (zijn eigenlijk gewoon de noise points)
for _, row in filtered_card_arrest_all.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip='Any Cardiac Problem'
    ).add_to(m)


# Add points to the map, colored by cluster
for _, row in filtered_card_arrest_all.iterrows():
    cluster_label = row['cluster']
    
    if cluster_label != -1:
        color = cluster_colors[cluster_label]
    
        folium.CircleMarker(
            location=[row['latitude_intervention'], row['longitude_intervention']],
            radius=1,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"Cluster: {cluster_label}"
        ).add_to(m)
        

# Save the map to an HTML file
m.save('dbscan_plus_card_arrest_test.html')


In [ ]:
#plot toont dat de clusters, met die settings, sowieso zeer klein zijn, en dat je eigenlijk gewoon de AED recht in het midden 
#ervan kan zetten. verder dan dat gaan is mogelijks overengineering. Maar moet ook kijken naar hoeveel instances er in de clusters aanwezig zijn,
#er moeten er sowieso meer zijn dan AED's beschikbaar.

#er zijn we nog altijd clusters die bestaan uit 1 instance om 1 of andere reden. dus die achteraf verwijderen, of in de code fixen

In [ ]:
#de oplossing waarbij er per provincie wordt gekeken: gewoon hetzelfde, alleen maak je een dataset voor de cardiac arrests EN de aed's die uit die
#provincie komen, en voor de rest doe je dezelfde analyse.

#kan de tweede clustering analyse doen op clusters die groter zijn dan 3 (of welke waarde dan ook), en voor die van 3 gewoon de centroid nemen.

In [300]:
#4242
num_clusters = len(set(labels) - {-1})
num_clusters

4242

In [307]:
with open('size_counts_output.txt', 'w') as f:
    f.write(size_counts.to_string())

In [310]:
#31517 out of 56284 get assigned to a cluster. gegeven dat we 11500 aeds hebben om te plaatsen, misschien wel ok eigenlijk.
#er zijn 1555 clusters van 3 (dus 4.665, bijna 15%), als we criterium verhogen naar 4 units per cluster, dan is zijn er 26.961 in een cluster.
#maar eigenlijk is er niet echt een reden om te verwijderen, behalve als de performance shit blijkt, dan kan je dat wel doen.
non_noise_count = np.sum(labels != -1)
non_noise_count

np.int64(31517)

In [ ]:
#clusters kleiner dan een bepaalde grootte, die sowieso maar 1 AED zouden krijgen, daar plaats je de AED gewoon op de centroid
#voor grotere waar meer dan 1 AED voor is voorbehouden, probeer je zoveel mogelijk oppervlakte te bedekken met je AED.

In [ ]:
#deze bevat alles.
#voor kleine clusters kan je gewoon de centroid telkens berekenen, en dat zijn de coordinaten van de AED.
filtered_card_arrest_all

In [319]:
total_aeds = aeds_to_move_with_locations.shape[0]
total_aeds

11624

In [324]:
# Calculate proportional AEDs for each cluster
# Remove noise points (label -1)
cluster_sizes = cluster_sizes[cluster_sizes.index != -1]

# Calculate proportional allocation
total_points_in_clusters = cluster_sizes.sum()
proportional_aeds = (cluster_sizes / total_points_in_clusters * total_aeds).round().astype(int)

coverage_radius = 217.5


In [325]:
from scipy.spatial import distance
import numpy as np

def calculate_centroid(df):
    """ Calculate centroid of a cluster """
    return df[['latitude_intervention', 'longitude_intervention']].mean().values

def place_aeds_centroid(cluster_df, num_aeds):
    """ Place AEDs at the centroid for smaller clusters """
    centroid = calculate_centroid(cluster_df)
    return [centroid] * num_aeds

def place_aeds_coverage(cluster_df, num_aeds):
    """ Place AEDs to maximize coverage for larger clusters """
    points = cluster_df[['latitude_intervention', 'longitude_intervention']].values
    centroids = []
    while len(centroids) < num_aeds and len(points) > 0:
        # Find point farthest from existing centroids (greedy approach)
        distances = np.min(distance.cdist(points, np.array(centroids) if centroids else points), axis=1)
        farthest_point = points[np.argmax(distances)]
        centroids.append(farthest_point)
        # Remove covered points
        points = points[distances > coverage_radius]
    return centroids

# Iterate through clusters and determine AED placement
results = []
for cluster_label in filtered_card_arrest_all['cluster'].unique():
    if cluster_label != -1:  # Skip noise
        cluster_df = filtered_card_arrest_all[filtered_card_arrest_all['cluster'] == cluster_label]
        num_aeds = proportional_aeds.get(cluster_label, 1)
        
        if len(cluster_df) < 5:  # Threshold for smaller clusters, moet nog gefinetuned worden
            aed_locations = place_aeds_centroid(cluster_df, num_aeds)
        else:
            aed_locations = place_aeds_coverage(cluster_df, num_aeds)
        
        results.append({
            'cluster': cluster_label,
            'aed_locations': aed_locations
        })

# Display results
for result in results:
    print(f'Cluster {result["cluster"]} - AED Locations: {result["aed_locations"]}')

C:\Users\eloua\AppData\Local\Temp\ipykernel_20008\1851617741.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_aeds = proportional_aeds.get(cluster_label, 1)
C:\Users\eloua\AppData\Local\Temp\ipykernel_20008\1851617741.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_aeds = proportional_aeds.get(cluster_label, 1)
C:\Users\eloua\AppData\Local\Temp\ipykernel_20008\1851617741.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[

Cluster 0 - AED Locations: [array([50.87798,  4.39229])]
Cluster 1 - AED Locations: [array([50.89578,  4.35817])]
Cluster 2 - AED Locations: [array([50.85592,  4.42592])]
Cluster 3 - AED Locations: [array([50.80938,  4.34743])]
Cluster 4 - AED Locations: [array([50.86944,  4.3608 ])]
Cluster 5 - AED Locations: [array([50.78859,  4.31543])]
Cluster 6 - AED Locations: [array([50.83808,  4.30484])]
Cluster 7 - AED Locations: [array([50.87307,  4.35711])]
Cluster 8 - AED Locations: [array([50.8248 ,  4.38185])]
Cluster 9 - AED Locations: [array([50.83848,  4.44441])]
Cluster 10 - AED Locations: [array([50.85879,  4.41331])]
Cluster 11 - AED Locations: [array([50.79383,  4.33399])]
Cluster 12 - AED Locations: [array([50.89141,  4.34222])]
Cluster 13 - AED Locations: [array([50.82453,  4.41241])]
Cluster 14 - AED Locations: [array([50.845  ,  4.29674])]
Cluster 15 - AED Locations: [array([50.83478,  4.29751])]
Cluster 16 - AED Locations: [array([50.81582,  4.34141])]
Cluster 17 - AED Locatio